In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data_train=pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding="latin1")
data_test=pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding='latin1')

In [ ]:
data_train.head()

In [ ]:
data_test.head()

# **Basic EDA 🤔**

In [ ]:
print(data_train['Sentiment'].unique())

# *Merging the extreme cases with their parent class*

In [ ]:
for i in range(0,len(data_train)):
    if(data_train['Sentiment'][i]=='Extremely Negative'):
        data_train['Sentiment'][i]='Negative'
    elif(data_train['Sentiment'][i]=='Extremely Positive'):
        data_train['Sentiment'][i]='Positive'

In [ ]:
print(data_train['Sentiment'].unique())

In [ ]:
data_train['Sentiment'].isna().sum()

#  *Let's convert the Sentiment Labels into numericals for easier processing*

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
data_train['Sentiment']=le.fit_transform(data_train['Sentiment'])

In [ ]:
##Converting into one hot encoding form is important when doing MultiClass Classification in Tensorflow
y=pd.get_dummies(data_train['Sentiment'])

In [ ]:
data_train.head()

In [ ]:
print(le.inverse_transform([0,1,2]))

In [ ]:
n=0
ne=0
p=0

In [ ]:
for i in range(0,len(data_train)):
    if(data_train['Sentiment'][i]==0):
        n=n+1
    elif(data_train['Sentiment'][i]==1):
        ne=ne+1
    elif(data_train['Sentiment'][i]==2):
        p=p+1
   

# *Printing the total data for each individual classes*

In [ ]:
print('Total Labels belonging to Negative Category are ',n)
print('Total Labels belonging to Neutral Category are ',ne)
print('Total Labels belonging to Positive Category are ',p)


In [ ]:
print('Total entries in the train dataset is ',len(data_train))

In [ ]:
##Segregating the tweet class from their original dataset
X=data_train.iloc[:,4]

In [ ]:
from sklearn.model_selection import train_test_split
train_text,validation_text,train_labels,validation_labels=train_test_split(X.to_numpy(),y.to_numpy(),test_size=0.2,random_state=42)

In [ ]:
##printing validation labels to check if they are in one hot encoded form or not
validation_labels

In [ ]:
type(train_text)

In [ ]:
import re
import string

# *Cleaning the textual data- Removing all Hashtags, Mentions, Punctuations and Links*

In [ ]:
for i in range(0,len(train_text)):
    temp=train_text[i]
    temp=re.sub("@\S+", " ", temp)
    temp=re.sub("https*\S+", " ", temp)
    temp=re.sub("#\S+", " ", temp)
    temp=re.sub("\'\w+", '', temp)
    temp=re.sub('[%s]' % re.escape(string.punctuation), ' ', temp)
    temp=re.sub(r'\w*\d+\w*', '', temp)
    temp=re.sub('\s{2,}', " ", temp)
    train_text[i]=temp
    ##print(temp)

In [ ]:
for i in range(0,len(validation_text)):
    temp=validation_text[i]
    temp=re.sub("@\S+", " ", temp)
    temp=re.sub("https*\S+", " ", temp)
    temp=re.sub("#\S+", " ", temp)
    temp=re.sub("\'\w+", '', temp)
    temp=re.sub('[%s]' % re.escape(string.punctuation), ' ', temp)
    temp=re.sub(r'\w*\d+\w*', '', temp)
    temp=re.sub('\s{2,}', " ", temp)
    validation_text[i]=temp
    ##print(temp)

# *Print some random strings to check if the data has been preprocessed and cleaned well*

In [ ]:
import random

In [ ]:
for i in range(0,10):
    print('New Tweet')
    print('***********')
    random_number=random.randint(0,len(train_text)-1)
    print(train_text[random_number])

In [ ]:
for i in range(0,10):
    print('New Tweet')
    print('***********')
    random_number=random.randint(0,len(validation_text)-1)
    print(validation_text[random_number])

In [ ]:
type(train_text)

# Using Tensorflow 2.x to create models 😍

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
sum_length_of_tweet=0

In [ ]:
##finding the average length of each tweet
for i in X:
    temp=i
    sum_length_of_tweet=sum_length_of_tweet+len(temp.split())
    

In [ ]:
max_length=round(sum_length_of_tweet/len(X))

In [ ]:
print(max_length)

In [ ]:
Tokenizer?

# *Tokenization and Padding the textual data*

In [ ]:
tokenizer=Tokenizer(num_words=10000,oov_token='</OOV>')

In [ ]:
len(tokenizer.word_index)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer.fit_on_texts(train_text)

In [ ]:
train_text_sequences=tokenizer.texts_to_sequences(train_text)

In [ ]:
train_text_padded=pad_sequences(train_text_sequences,maxlen=max_length,padding='post')

In [ ]:
validation_text_sequences=tokenizer.texts_to_sequences(validation_text)

In [ ]:
validation_text_padded=pad_sequences(validation_text_sequences,maxlen=max_length,padding='post')

# *Let's print some examples to check if it functions the way we expect it to work or not*

In [ ]:
for i in range(0,10):
    index=random.randint(0,len(train_text)-1)
    print('Original Sentence')
    print('*******')
    print(train_text[index])
    print('Padded Sentence')
    print('*********')
    print(train_text_padded[index])

In [ ]:
for i in range(0,10):
    index=random.randint(0,len(validation_text)-1)
    print('Original Sentence')
    print('*******')
    print(validation_text[index])
    print('Padded Sentence')
    print('*********')
    print(validation_text_padded[index])

In [ ]:
train_text_padded.shape[1]

# a) Making Baseline Model

In [ ]:
model_1=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000,128,input_length=train_text_padded.shape[1]),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
history_1=model_1.fit(train_text_padded,train_labels,epochs=4,validation_data=(validation_text_padded,validation_labels))

# <span style=color:red>*Baseline Model looks good. It gives us an accuracy of 81% on the validation dataset*</span>

# b) Improving the baseline Model

In [ ]:
model_1.evaluate(validation_text_padded,validation_labels)

In [ ]:
model_2=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000,128,input_length=train_text_padded.shape[1]),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(8,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])
model_2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
history_2=model_2.fit(train_text_padded,train_labels,epochs=4,validation_data=(validation_text_padded,validation_labels))

# c) LSTM based Model

In [ ]:
model_3=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000,128,input_length=max_length),
    tf.keras.layers.LSTM(8),
    tf.keras.layers.Dense(8,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])
model_3.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
history_3=model_3.fit(train_text_padded,train_labels,epochs=3,validation_data=(validation_text_padded,validation_labels))

# <span style=color:red>*Accuracy improves to ~85% on the validation dataset while using LSTM Based Architecture*</span>

# d) Improving the LSTM based model even further

In [ ]:
model_4=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000,128,input_length=train_text_padded.shape[1]),
    tf.keras.layers.LSTM(512),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])
model_4.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
history_4=model_4.fit(train_text_padded,train_labels,epochs=4,validation_data=(validation_text_padded,validation_labels))

# <span style=color:red>*Again close to 85% on the validation dataset. Looks good*</span>

In [ ]:
len(train_text_padded[0])

In [ ]:
model_4.summary()

# e) Trying Universal Sentence Encoder from TensorHub.

In [ ]:
import tensorflow_hub as hub


In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
train_embed=embed(train_text)
test_embed=embed(validation_text)

In [ ]:
len(train_embed[0])

In [ ]:
model_6=tf.keras.models.Sequential([
    tf.keras.layers.Dense(3,activation='softmax')
])
model_6.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model_6.fit(train_embed,train_labels,epochs=30,validation_data=(test_embed,validation_labels))

# f) Final attempt: Bidirectional LSTM based Architecture

In [ ]:
model_new=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000,128,input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])
model_new.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model_new.fit(train_embed,train_labels,epochs=5,validation_data=(test_embed,validation_labels))

# *We got our best result from Model 4 ie Improved LSTM architecture. We will be using the same model on our test dataset*

In [ ]:
data_test

In [ ]:
data_test.iloc[:,4]

In [ ]:
for i in range(0,len(data_test)):
    if(data_test['Sentiment'][i]=='Extremely Negative'):
        data_test['Sentiment'][i]='Negative'
    elif(data_test['Sentiment'][i]=='Extremely Positive'):
        data_test['Sentiment'][i]='Positive'

In [ ]:
X_test=data_test.iloc[:,4]

In [ ]:
##preprocessing the test dataset
for i in range(0,len(X_test)):
    temp=X_test[i]
    temp=re.sub("@\S+", " ", temp)
    temp=re.sub("https*\S+", " ", temp)
    temp=re.sub("#\S+", " ", temp)
    temp=re.sub("\'\w+", '', temp)
    temp=re.sub('[%s]' % re.escape(string.punctuation), ' ', temp)
    temp=re.sub(r'\w*\d+\w*', '', temp)
    temp=re.sub('\s{2,}', " ", temp)
    X_test[i]=temp
    ##print(temp)

In [ ]:
final_result=[]

In [ ]:
for i in range(0,len(X_test)):
    text=[X_test[i]]
    text=tokenizer.texts_to_sequences(text)
    text=pad_sequences(text,maxlen=max_length,padding='post')
    result=model_4.predict_classes(text)
    if(result==0):
        final_result.append('Negative')
    elif(result==1):
        final_result.append('Neutral')
    elif(result==2):
        final_result.append('Positive')

In [ ]:
actual_result=data_test.iloc[:,5]

In [ ]:
type(actual_result)

# Plotting the confusion matrix to check for accuracy.

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(actual_result,final_result)

# *We have got more than 83% accuracy on the test dataset which is fabulous. Kudos for sticking around 🤝*